# UI to conduct a manual audit of the labels from conventions classifiers. 


# ***IF NOT USING BINDER - UNCHECK THE BOX***


The output of the classifiers are stored to

    -file 'audit_training_data.gz (which is a zipped pandas database) 
    -location: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/

The UI randomly displays an already classified sentence for the purpose of audit and correction

Completed audits will be stored locally until the "Save to GIT" button is selected, in which case the tsv of audit will be stored to 

    - file 'audit_training_data-{gituser}{date}.tsv 
    -location: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/

Content of the Audited files may be coccat and added to the Train_Validate data.


In [1]:
import os
import re
import random
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display, clear_output, HTML, Image
import base64
import io
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import base64
from github import Github, GithubException
from github import InputGitTreeElement
import requests
import time
from io import BytesIO
from PIL import Image
import urllib.request

In [2]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
FLAG__ON_BINDER = True
FLAG__FROM_GIT = True

my_style = {'description_width': 'initial'}
my_layout = {'width': '600px'}
my_layout_short = {'width': '300px'}
my_layout_shortest = {'width': '250px'}

In [4]:

box_binder = widgets.Checkbox(True, description='*MUST un-check thid box IF NOT running on binder', style = my_style, layout = my_layout)

def update_box_binder(change):
    global FLAG__ON_BINDER
    global FILE_PATH
    global DOWNLOAD_PATH
    FILE_NAME = 'audit_training_data.gz'
    FLAG__ON_BINDER = change['new']
    
    if (FLAG__ON_BINDER == True):
        DOWNLOAD_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
        DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
        FILE_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'

    else:
        DOWNLOAD_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
        DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
        FILE_PATH = './Data/Iterative-models-building/Classification results/Conventions/Audited/'
        if not os.path.exists(DOWNLOAD_PATH):
            os.makedirs(DOWNLOAD_PATH)

        if not os.path.exists(FILE_PATH):
            os.makedirs(FILE_PATH)
    print (FILE_PATH)
   
    
box_binder.observe(update_box_binder, 'value')

display(box_binder)
print("....loading test tool")

Checkbox(value=True, description='*MUST un-check thid box IF NOT running on binder', layout=Layout(width='600p…

....loading test tool


In [5]:
FILE_NAME = 'audit_training_data.gz'

TOTAL_RECORDS_AUDITED = 0
TOTAL_UNSAVED_AUDITS = 0
LAST_SAVED = ""

if FLAG__ON_BINDER == True:
    DOWNLOAD_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
    DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
    FILE_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'
    
else:
    DOWNLOAD_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
    DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
    FILE_PATH = './Data/Iterative-models-building/Classification results/Conventions/Audited/'
    
if FLAG__FROM_GIT == True:
    DOWNLOAD_PATH = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/audit_training_data.gz?raw=true'
    GIT_PUSH_PATH = 'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/' 
    IMAGE_PATH = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/convention_training_data_proportions.png'
    IMG =  'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/histogram.png'

In [6]:
def image_to_byte_array(image:Image):
  imgByteArr = io.BytesIO()
  image.save(imgByteArr, format=image.format)
  imgByteArr = imgByteArr.getvalue()
  return imgByteArr

def retrieve_data_frame ():
   
    if FLAG__FROM_GIT == True:
        # sep='\t'  to specify tab seperated?
        df = pd.read_csv(DOWNLOAD_PATH, error_bad_lines=False, compression='gzip', header=0, quotechar='"')

    if FLAG__FROM_GIT == False:
        df = pd.read_csv(DOWNLOAD_PATH, sep='\t')

    df = df.drop(df.columns[0], axis=1)
    
    #print("num of records with NaN", df.isna().sum() )
    
    #Drop any rows where there is 0 confidence value. 
    df = df.dropna(subset=['confidence_value'])
    #if source is not specified, fill as other. 
    df['data_provenance'] = df.data_provenance.fillna('other')
    row_to_edit = df.sample(n = 1)
    index_number = 0     
    #print("num of records with NaN", df.isna().sum() )   
    #remove rows where text = "unknown"
    df = df[df['text']!="unknown"]
    
    print("data file loaded:", len(df.index), " rows")
    return df


In [17]:
# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '15px'),
  ('text-align', 'center'),
  #('font-weight', 'bold'),
  ('color', 'grey'),
  ('background-color', 'lightgrey'),
  ('border-color', 'black'),
  ('border-style' ,'solid') ,
  ('border-width', '1px'),
  ('border-collapse','collapse'),
  ('max-width', '700px'),
  ('min-width', '700px')
  ]
  

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '15px'),
  ('text-align', 'left'),
  #('font-weight', 'bold'),
  ('color', 'lightgrey'),
  ('background-color', 'grey'),
  ('border-color', 'black'),
  ('border-style' ,'solid') ,
  ('border-width', '1px'),
  ('border-collapse','collapse'),
  ('height', '100px'),
  ('max-width', '700px'),
  ('min-width', '700px')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]


In [8]:
convention_sentences_df = []

audited_df = pd.DataFrame(columns=['text', 'provenance', 'convention', 'old', 'new'])
convention_sentences_df = retrieve_data_frame ()

urlBase  = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/VALIDATED_DATA/Conventions/keywords/keywords.csv'
dfKeyWords = pd.read_csv(urlBase)
list_conv = pd.unique(dfKeyWords['convention'])
keyword_arr = {}

for conv in list_conv:
    keyword_arr[conv] = dfKeyWords.loc[dfKeyWords['convention'] == conv, 'keyword'].tolist()
    


data file loaded: 206107  rows


In [9]:
min_confidence = convention_sentences_df.confidence_value.min()
if min_confidence < .01:
    min_confidence = 0

max_confidence = convention_sentences_df.confidence_value.max()
print ("Data to audit loaded from GitHub repo")

Data to audit loaded from GitHub repo


In [10]:
def time_stamp():
    now = datetime.now()
    timestamp = datetime.timestamp(now)
    dt_object = datetime.fromtimestamp(timestamp)
    words = str(dt_object).split(' ');
    return words[0], words[1]

def save_to_github(git_user, git_password, git_repo, my_file_list, push_to_git_as):
    '''
    in order to push a file to github it must first be stored locally, then pushed
    this local location can also be local to a virtual machine. 
    takes: 
            git username, password, repo, 
            a list of files to push to git ie the full local location of file,
            a matching list of paths to push each file to in Git hub 
    '''
    user = git_user
    password = git_password
    url = git_repo
    file_list = []  #push these list of files to git
    file_names = [] #push to this location in git
    message = 'ok'
    
    
    try:
        g = Github(user, password)
        try:
            repo = g.get_user().get_repo(url)
        except (IOError, OSError, GithubException) as e:
            
            return "error", e.message

        file_list = my_file_list 
        file_names = push_to_git_as
                 
        commit_message = 'training data updated via the audit tool'
    
        master_ref = repo.get_git_ref('heads/master')
        master_sha = master_ref.object.sha
        base_tree = repo.get_git_tree(master_sha)
        element_list = list()

        for i, entry in enumerate(file_list):   
            with open(entry) as input_file:
                #data = input_file.read()   #works with non zip file
                data = base64.b64encode(open(entry, "rb").read())
                
            if entry.endswith('.png'):
                data = base64.b64encode(data) 
                

            blob = repo.create_git_blob(data.decode("utf-8"), "base64")
            element = InputGitTreeElement(path=file_names[i], mode='100644', type='blob', sha=blob.sha)

            #push to git as file_names[i]
            #print ("push to git as:", file_names[i])           
            #element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
        
        
            #element_list is a list of InputGitTreeElement. 
            #Each one corresponds to a file. 
            # the 'content' of InputGitTreeElement can only be of type 'str' or 'unicode'. 
            #When I load a file to memory I have type 'bytes'. 
            #What is the right way to encode those bytes to str or unicode to upload a .zip
            element_list.append(element)   
        
        tree = repo.create_git_tree(element_list, base_tree)
        parent = repo.get_git_commit(master_sha)
        commit = repo.create_git_commit(commit_message, tree, [parent])
        master_ref.edit(commit.sha)
        return commit, message 
    except e:
        message = "GitHub save FAILED:" + '\n' +"Are your github login credentials correct?" + '\n' + "Are you a collaberator in the repo?"
        return "error", message


def save_to_github_not_zip(git_user, git_password, git_repo, my_file_list, push_to_git_as):
    user = git_user
    password = git_password
    url = git_repo
    file_list = []  #push these list of files to git
    file_names = [] #push to this location in git
    message = "ok"
    
    try:
        g = Github(user,password)

        try:
            repo = g.get_user().get_repo(url)
        except (IOError, OSError, GithubException) as e:
            return "error", e.message

        file_list = my_file_list 
        file_names = push_to_git_as
                 
        commit_message = 'training data audited via the audit tool'
    
        master_ref = repo.get_git_ref('heads/master')
        master_sha = master_ref.object.sha
        base_tree = repo.get_git_tree(master_sha)
        element_list = list()

        for i, entry in enumerate(file_list):
            with open(entry) as input_file:
                data = input_file.read()
            if entry.endswith('.png'):
                data = base64.b64encode(data)
                
            #print ("file to commit:", entry)
            #print ("push to git as:", file_names[i])
            element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
            element_list.append(element)
            
        tree = repo.create_git_tree(element_list, base_tree)
        parent = repo.get_git_commit(master_sha)
        commit = repo.create_git_commit(commit_message, tree, [parent])
        master_ref.edit(commit.sha)
        return commit, message
    except:
        
        message = "GitHub save FAILED:" + '\n' +"Are your github login credentials correct?" + '\n' + "Are you a collaberator in the repo?"
        return "error", message
        


In [18]:
selectHTMLHead = widgets.HTML(
            "<font color = '#8c8c8c'><h3><center>*** Model Classification, Audit Tool ***</center></h3></font>")

selectHTML = widgets.HTML(
            "<font color = '#8c8c8c'><h4><left>--- Filter ---</left></h4></font>")

saveHTML = widgets.HTML(
            "<h4><left><font color = '#8c8c8c'>--- Push unsaved audits to Github repo ---</font></left></h4>")

actionHTML = widgets.HTML(
            "<font color = '#8c8c8c'><h4><left>--- Action ---</h4></left></font>")


space = widgets.Label('  ', layout=widgets.Layout(width='100%'))



convention_drop_down = widgets.Dropdown(
    options={ 
             'Civic':'convention_civic', 
             'Domestic':'convention_domestic', 
             'Green':'convention_green', 
             'Industrial':'convention_industrial', 
             'Inspired':'convention_inspired',
             'Market':'convention_market', 
             'Project':'convention_project',
             'Renown':'convention_renown'},
    value='convention_civic',
    description='Convention  :',
    disabled=False,
    style = my_style,
    layout = my_layout
)

source_drop_down = widgets.Dropdown(
    options =  convention_sentences_df.data_provenance.unique(),
    description="Data source :",
     #value=None,
    disabled=False,
    style = my_style,
    layout = my_layout
)



reclassify_drop_down = widgets.Dropdown(
    options={'Negative for selected convention':'False Pos', 
             'Negative for selected and Positive for CIVIC':'convention_civic', 
             'Negative for selected and Positive for DOMESTIC':'convention_domestic', 
             'Negative for selected and Positive for GREEN':'convention_green', 
             'Negative for selected and Positive for INDUSTRIAL':'convention_industrial', 
             'Negative for selected and Positive for INSPIRED':'convention_inspired',
             'Negative for selected and Positive for MARKET':'convention_market', 
             'Negative for selected and Positive for PROJECT':'convention_project',
             'Negative for selected and Positive for RENOWN':'convention_renown'},
    value='False Pos',
    description='Change to:',
    disabled=False,
    style = my_style,
    layout = my_layout_short
)



convention_true_false = widgets.Dropdown(
    options=['True', 'False'],
    value='True',
    description='Logged as:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

confidence_score_slider_min = widgets.FloatSlider(
    min=min_confidence, 
    max=max_confidence,
    value = 0,
    step = 0.1,
    description='Min Confidence level:',
    style = my_style,
    layout = my_layout
    )
confidence_score_slider_min.style.handle_color = '#5c85d6'


confidence_score_slider_max = widgets.FloatSlider(
    min=min_confidence, 
    max=max_confidence, 
    value = max_confidence,
    step = 0.1,
    description='Max Confidence level:',
    style = my_style,
    layout = my_layout
    )
confidence_score_slider_max.style.handle_color = '#5c85d6'

change_value_button = widgets.Button(
            description='Change(False Pos)',
            disabled=False,
            button_style='',
            tooltip='Audit complete, change classification',
        )
change_value_button.style.button_color = '#e68a00'


keep_value_button = widgets.Button(
            description='OK(True Pos)',
            disabled=False,
            button_style='',
            tooltip='Audit complete, already classified ok',

        )
keep_value_button.style.button_color = '#00b386'


audit_next_button = widgets.Button(
            description='Skip',
            disabled=False,
            button_style='',
            tooltip='Move to next, without audit'
        )
audit_next_button.style.button_color = '#d9d9d9'

save_to_git_button = widgets.Button(
            description='Save to GitHub',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )
save_to_git_button.style.button_color = '#00b386'



user = widgets.Text(
            description='Username',
            disabled=False,
            layout=my_layout_shortest,
            tooltip='Enter your GitHub username',
            style = my_style
        )


pswd = widgets.Password(
            description='Password',
            disabled=False,
            layout=my_layout_shortest,
            tooltip='Enter your GitHub password',
            style = my_style
        )


def change_value_button_clicked(b):
    global TOTAL_UNSAVED_AUDITS 
    global TOTAL_RECORDS_AUDITED 
    global index_number
    global row_to_edit
    global audited_df
    global convention_sentences_df
    
    current_value = row_to_edit[convention_drop_down.value].values.astype(int)[0]
    if (current_value == 1) :
        new_value = 0
        
    elif (current_value == 0):
        new_value = 1
            
    text = row_to_edit.text.values.astype(str)[0]
    prov = row_to_edit.data_provenance.values.astype(str)[0]
    #current_conv is = to the convention being audited
    current_conv = convention_drop_down.value.replace('convention_', '')
    
    #if the classification is to register a False positive and a new classification i.e 1 to 0 and 0 to 1
    if (reclassify_drop_down.value != 'False Pos'):
        new_conv = reclassify_drop_down.value.replace('convention_', '')
        new_value = current_value
        current_value = 0
    
        current = pd.DataFrame({
                            'text': [text],
                            'provenance': [prov],
                            'convention': [new_conv],
                            'old': [current_value],
                            'new': [new_value]
                            })
        
        audited_df = audited_df.append(current, ignore_index=True)
    
        current = pd.DataFrame({
                            'text': [text],
                            'provenance': [prov],
                            'convention': [current_conv],
                            'old': [new_value],
                            'new': [current_value]
                            })
        
        audited_df = audited_df.append(current,  ignore_index=True)
    
    else:#the classification is to register a False positive
    
        current = pd.DataFrame({
                            'text': [text],
                            'provenance': [prov],
                            'convention': [current_conv],
                            'old': [current_value],
                            'new': [new_value]
                            })
        audited_df = audited_df.append(current,  ignore_index=True)
    
    
    TOTAL_UNSAVED_AUDITS = TOTAL_UNSAVED_AUDITS + 1
    TOTAL_RECORDS_AUDITED = TOTAL_RECORDS_AUDITED + 1
    with output:
        clear_output(wait=True)
        output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value )
    
    with outputToSave:
        clear_output(wait=True)
        display (audited_df)
        
    with outputGitHub:
        clear_output(wait=True)
        print ("")
        
            
def keep_value_button_clicked(b):
    global row_to_edit
    global audited_df
    global TOTAL_UNSAVED_AUDITS 
    global TOTAL_RECORDS_AUDITED 
    with output:
        clear_output(wait=True)
        text = row_to_edit.text.values.astype(str)[0]
        prov = row_to_edit.data_provenance.values.astype(str)[0]
        conv = convention_drop_down.value.replace('convention_', '')
        current = pd.DataFrame({'text': [text],
                                'provenance': [prov],
                                'convention': [conv],
                                'old': [1],
                                'new': [1]
                               })
        audited_df = audited_df.append(current,ignore_index=True)
        TOTAL_UNSAVED_AUDITS = TOTAL_UNSAVED_AUDITS + 1
        TOTAL_RECORDS_AUDITED = TOTAL_RECORDS_AUDITED + 1
        output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value )
    with outputToSave:
        clear_output(wait=True)
        display (audited_df)
    with outputGitHub:
        clear_output(wait=True)
        print("")
        

def audit_next_button_clicked(b):
    with output:
        clear_output(wait=True)
        output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value )
    with outputToSave:
        clear_output(wait=True)
        display (audited_df)
    with outputGitHub:
        clear_output(wait=True)
        print("")

def save_changes_to_github_button_clicked(b):
    global TOTAL_UNSAVED_AUDITS 
    global convention_sentences_df
    global LAST_SAVED
    global FILE_PATH
    global GIT_PUSH_PATH
    global audited_df
    GIT_USER = user.value
    GIT_PSWD = pswd.value
    GIT_REPO = 'AIVC'
    D, T = time_stamp()
    file_name = 'audited_training_data_'+ GIT_USER   + '_' + D+T +  '.tsv'
    file_name_zip = 'audit_training_data_'+ GIT_USER + '_' + D+T +   '.gz'
    
    try:
        audited_df.to_csv(FILE_PATH + file_name, sep = '\t', index = False)
    except (IOError, OSError) as e:
        with outputGitHub:
            clear_output(wait=True)
            display(HTML("<font color='red'><b>Warning!!</b></font>"))
            print ("Are your settings correct for local machine v's binder?")
            print (e)
        return "error"
        
    my_file_list = [FILE_PATH + file_name]
    push_to_git_as = [GIT_PUSH_PATH + file_name]
    commit, message = save_to_github_not_zip(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)
    if (commit != "error"):
        with outputGitHub:
            clear_output(wait=True)
            print ("File to commit:", my_file_list[0])
            print ("Push to git as:", push_to_git_as[0])
            print ("Commit: ", commit)
            TOTAL_UNSAVED_AUDITS = 0
            D, T = time_stamp()
            LAST_SAVED = D+T
            print ("Number of audits saved: ", audited_df.shape[0] )
            display(HTML("<font color='green'><b>Saved!!</b></font>"))
            audited_df = pd.DataFrame(columns=['text', 'provenance', 'convention', 'old', 'new'])
    if (commit == "error"):
        with outputGitHub:
            clear_output(wait=True)
            print ("File to commit: ", my_file_list[0])
            print ("Push to git as: ", push_to_git_as[0])
            display(HTML("<font color='red'><b>Warning!!</b></font>"))
            print(message)
            

def output_function(convention_drop_down_value, confidence_score_slider_min_value, confidence_score_slider_max_value, source_drop_down_value ):
    global convention_sentences_df
    global row_to_edit
    global index_number
    global LAST_SAVED
    global TOTAL_RECORDS_AUDITED
    global TOTAL_UNSAVED_AUDIT
    
    reclassify_drop_down.value='False Pos'
    convention = convention_drop_down_value.replace('convention_', '')

   
    try:
        row_to_edit = convention_sentences_df.loc[(convention_sentences_df[convention_drop_down_value] == 1) &
                                            (convention_sentences_df['data_provenance'] == source_drop_down_value) &
                                            (convention_sentences_df['confidence_value'] >= confidence_score_slider_min_value) &
                                            (convention_sentences_df['confidence_value'] <= confidence_score_slider_max_value)].sample(n = 1)  

    except ValueError:
       
        display(HTML("<font color='red'><b>Warning</b> " + "No data, please adjust filter:</font>"))
    
    if not row_to_edit.empty:
        row_to_edit = row_to_edit[['text', convention_drop_down_value, 'confidence_value', 'data_provenance']]
        display (row_to_edit[['text']].style.background_gradient().set_caption('Row randomly selected based on filter').set_table_styles(styles).hide_index())
        sentence = row_to_edit['text'].values.astype(str)[0]
        conf = str(row_to_edit['confidence_value'].values.astype(float)[0])
        #display(HTML("Sentence: <b><font color='green'>" + sentence + "</b></font>" ))
        display(HTML("Text field classified as: <b><font color='black'>" + "POSITIVE </b> " + "</font> for the <font color='black'><b>" +  convention.upper() + " </b></font>economy of conventions, with confidence </> "+conf ))
        index_number = row_to_edit.index.values.astype(int)[0]
        try:
            with outputCount:
                clear_output(wait=True)
                print("Total audited:", TOTAL_RECORDS_AUDITED, " Total unsaved:", TOTAL_UNSAVED_AUDITS," Last Saved:", LAST_SAVED )
        except:
            print ("")
    else:
        print ("No results, change search setting")
    
    
  
output = widgets.interactive_output(output_function, {
            'convention_drop_down_value': convention_drop_down,
            'confidence_score_slider_min_value': confidence_score_slider_min,
            'confidence_score_slider_max_value': confidence_score_slider_max,
            'source_drop_down_value': source_drop_down
            })

outputGitHub = widgets.Output()
outputToSave = widgets.Output()
outputCount = widgets.Output()
change_value_button.on_click(change_value_button_clicked)
keep_value_button.on_click(keep_value_button_clicked)
audit_next_button.on_click(audit_next_button_clicked)
save_to_git_button.on_click(save_changes_to_github_button_clicked)


In [12]:
#HTML Help Content   
using = '<h4>Using the audit tool</h4><p>This tool will randomly select and display a sentence which has already been classified by the model thus facilitating human evaluation and correction of model predictions. These validated and corrected predictions will be fed back to the model as part of a re-training process, with the objective of increasing model accuracy. The tool will automatically select and display a random sentence from the pool each time an action is involked, either by modifying the filter or by one of the three action buttons [Change], [Ok] and [Skip]. Once the sentence is audited via either the [Ok] or [Change] button it will be used to train the model on the next training iteration.<br/>[Skip]: Use the Skip button if you are unable to confirm the classification (no action will be taken)</p><p>[Ok]: </span> Use the OK button to confirm that the sentence has been classified correctly as [Positive] for the convention in question i.e if the classification is a "True Positive" </p><p>[Chang]e]: Use the change button, if you believe the sentence has been classified incorrectly by the model  i.e if the classification is a "False Positive".<br/> If you can correctly classify the sentence as belonging to another convention, please select the appropriateconvention via the [Change to] dropdown.</p><p><em>Note: A&nbsp;<strong>GitHub</strong>&nbsp;username and password is required to save the results of the audits, it is important to remember to save the audit on a regular basis</em></p><hr/><p></p>'                              
filtering = '<h4>Filtering</h4>To help obtain a more balanced training data set across all convention models, please review the histogram to determine focus of audit'

#Histogram image
image = Image.open(urllib.request.urlopen(IMAGE_PATH))
image = image_to_byte_array(image)

#keywords tab
tab_contents = keyword_arr.values()
tab_name = list(keyword_arr.keys())
children = [widgets.HTML(value=str(name))for name in tab_contents]
tab_keywords = widgets.Tab()
tab_keywords.children = children
for i in range(len(children)):
    tab_keywords.set_title(i, str(tab_name[i]))

#SET UP A HEADER ACCORDIAN TO SHOW HELP, HISTOGRAM and KEYWORDS
accordion = widgets.Accordion(children=
                              [widgets.HTML(filtering+using),
                               widgets.Image(value=image,format='png',width=500,height=500),
                               tab_keywords
                              ]
                         
                             )


accordion.set_title(0, 'About The Tool')
accordion.set_title(1, 'Training Data Histogram')
accordion.set_title(2, 'Convention Keyword hints')
accordion.selected_index = None

In [13]:


display (
        selectHTMLHead,
        accordion,
        space,
        selectHTML,
        widgets.VBox([
        convention_drop_down, 
        source_drop_down, 
        confidence_score_slider_min, 
        confidence_score_slider_max]),
        output,
        space,
        actionHTML,
        reclassify_drop_down,
        widgets.HBox([change_value_button, keep_value_button, audit_next_button]),
        outputCount,
        space,
        saveHTML,
        space,
        widgets.HBox([user, pswd]),
        save_to_git_button,
        space,
        outputGitHub,
        outputToSave,
        space
    )

HTML(value="<font color = '#8c8c8c'><h3><center>*** Model Classification, Audit Tool ***</center></h3></font>"…

Accordion(children=(HTML(value='<h4>Filtering</h4>To help obtain a more balanced training data set across all …

Label(value='  ', layout=Layout(width='100%'))

HTML(value="<font color = '#8c8c8c'><h4><left>--- Apply Filter ---</left></h4></font>")

Output()

...

Button(description='Save to GitHub', style=ButtonStyle(button_color='#00b386'), tooltip='Click me')

Label(value='  ', layout=Layout(width='100%'))

Output()

Output()

Label(value='  ', layout=Layout(width='100%'))